Merging the e.coli and weather data 

Loading the data

In [63]:
import pandas as pd

# Load data
beach_data = pd.read_parquet("..\\data\\cleaned\\cleaned_hanlans_point_beach.parquet")
weather_data = pd.read_parquet("..\\data\\cleaned\\cleaned_toronto_city.parquet")

print(f"beach data shape: {beach_data.shape}")
print(f"beach data col. names: {beach_data.columns.to_list()}")

print(f"weather data shape: {weather_data.shape}")
print(f"weather data col. names: {weather_data.columns.to_list()}")

print(beach_data['e_coli_cfu'].head(5))
print(weather_data.head(5))


beach data shape: (1567, 5)
beach data col. names: ['beach_id', 'beach_name', 'e_coli_cfu', 'water_quality_advisory', 'safety_status']
weather data shape: (6575, 2)
weather data col. names: ['station_name', 'precip_mm']
date
2008-08-23     10.0
2008-08-24     10.0
2008-08-25     70.0
2008-08-26    159.0
2008-08-27     23.0
Name: e_coli_cfu, dtype: float64
            station_name  precip_mm
date                               
2007-01-01  TORONTO CITY        0.0
2007-01-02  TORONTO CITY        0.0
2007-01-03  TORONTO CITY        0.0
2007-01-04  TORONTO CITY        3.2
2007-01-05  TORONTO CITY        6.4


Split beach data by year

In [64]:
annual_beach = {
    year: group 
    for year, group in beach_data.groupby(beach_data.index.year)
}
print(annual_beach.keys())
print(annual_beach[2009].head(4))

dict_keys([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024])
            beach_id            beach_name  e_coli_cfu  \
date                                                     
2009-05-31       3.0  Hanlan's Point Beach        10.0   
2009-06-01       3.0  Hanlan's Point Beach        10.0   
2009-06-02       3.0  Hanlan's Point Beach        11.0   
2009-06-03       3.0  Hanlan's Point Beach        11.0   

                                       water_quality_advisory safety_status  
date                                                                         
2009-05-31  Beach safe. E.coli levels at or below 100 per ...          SAFE  
2009-06-01  Beach safe. E.coli levels at or below 100 per ...          SAFE  
2009-06-02  Beach safe. E.coli levels at or below 100 per ...          SAFE  
2009-06-03  Beach safe. E.coli levels at or below 100 per ...          SAFE  


Align weather data with 7-day lookback 

In [65]:
annual_weather = {}
for year, beach_df in annual_beach.items():
    # Get date range from beach data index
    start_date = beach_df.index.min() - pd.Timedelta(days=7)  # 7-day lookback
    end_date = beach_df.index.max()
    
    # Filter weather data (assuming it also has DateTimeIndex)
    weather_mask = (weather_data.index >= start_date) & (weather_data.index <= end_date)
    annual_weather[year] = weather_data.loc[weather_mask].copy()
    
    print(f"{year}: Weather from {start_date.date()} to {end_date.date()} ({len(annual_weather[year])} rows)")

2008: Weather from 2008-08-16 to 2008-08-31 (16 rows)
2009: Weather from 2009-05-24 to 2009-09-06 (106 rows)
2010: Weather from 2010-05-23 to 2010-09-07 (108 rows)
2011: Weather from 2011-05-05 to 2011-09-05 (124 rows)
2012: Weather from 2012-05-15 to 2012-09-03 (112 rows)
2013: Weather from 2013-05-14 to 2013-09-02 (112 rows)
2014: Weather from 2014-05-13 to 2014-09-01 (112 rows)
2015: Weather from 2015-05-12 to 2015-09-07 (119 rows)
2016: Weather from 2016-05-17 to 2016-09-06 (113 rows)
2017: Weather from 2017-05-16 to 2017-09-04 (112 rows)
2018: Weather from 2018-05-15 to 2018-09-03 (112 rows)
2019: Weather from 2019-05-14 to 2019-09-01 (111 rows)
2020: Weather from 2020-06-09 to 2020-09-07 (91 rows)
2021: Weather from 2021-05-18 to 2021-09-06 (112 rows)
2022: Weather from 2022-05-17 to 2022-09-05 (112 rows)
2023: Weather from 2023-05-16 to 2023-09-10 (118 rows)
2024: Weather from 2024-05-14 to 2024-09-02 (112 rows)


Merge data

In [68]:
annual_merged = {}
for year in annual_beach.keys():
    # Merge on index (date)
    merged = annual_beach[year].merge(
        annual_weather[year],
        left_index=True,
        right_index=True,
        how='left',
        suffixes=('_beach', '_weather')
    )
    
    # Add lagged weather features
    for lag in [1,2,3,5,7]:
        merged[f'precip_lag_{lag}d'] = merged['precip_mm'].shift(lag)
    
    annual_merged[year] = merged.dropna(subset=['e_coli_cfu', 'precip_lag_7d'])
    print(f"{year}: {len(merged)} rows")

2008: 9 rows
2009: 96 rows
2010: 96 rows
2011: 100 rows
2012: 98 rows
2013: 97 rows
2014: 101 rows
2015: 110 rows
2016: 106 rows
2017: 103 rows
2018: 103 rows
2019: 98 rows
2020: 66 rows
2021: 85 rows
2022: 99 rows
2023: 105 rows
2024: 95 rows


In [69]:
annual_merged[2009].head()

,beach_id,beach_name,e_coli_cfu,water_quality_advisory,safety_status,station_name,precip_mm,precip_lag_1d,precip_lag_2d,precip_lag_3d,precip_lag_5d,precip_lag_7d
date,,,,,,,,,,,,
2009-06-07,3.0,Hanlan's Point Beach,10.0,Beach safe. E.coli levels at or below 100 per ...,SAFE,TORONTO CITY,0.0,0.0,0.0,0.0,0.0,0.0
2009-06-08,3.0,Hanlan's Point Beach,73.0,Beach safe. E.coli levels at or below 100 per ...,SAFE,TORONTO CITY,8.0,0.0,0.0,0.0,0.0,0.0
2009-06-09,3.0,Hanlan's Point Beach,84.0,Beach safe. E.coli levels at or below 100 per ...,SAFE,TORONTO CITY,0.0,8.0,0.0,0.0,0.0,0.0
2009-06-10,3.0,Hanlan's Point Beach,24.0,Beach safe. E.coli levels at or below 100 per ...,SAFE,TORONTO CITY,0.0,0.0,8.0,0.0,0.0,0.0
2009-06-11,3.0,Hanlan's Point Beach,20.0,Beach safe. E.coli levels at or below 100 per ...,SAFE,TORONTO CITY,2.6,0.0,0.0,8.0,0.0,0.0


Write merged data to file

In [71]:
for year, df in annual_merged.items():
    df.to_parquet(f"../data/merged/merged_{year}.parquet")